In [115]:
import scipy.stats as sps
import numpy as np
import math

In [116]:
alpha = 0.05

# Критерий знаков

In [117]:
def sign_test_small(x, y):
    minus, plus = 0, 0
    for (x_i, y_i) in zip(x, y):
        if x_i < y_i: minus += 1
        elif x_i > y_i: plus += 1
    nk = minus + plus
    w = min(minus, plus)
    print(f"Число '-': {minus}, число '+': {plus}")

    t = 2 ** (-nk) * sum([math.comb(nk, j) for j in range(w + 1)])

    print(f"Статистика t = {t}, интервал [{- alpha / 2}, {1 - alpha / 2}]")
    if - alpha / 2 < t < 1 - alpha / 2: print('Средние выборок равны')
    elif - alpha / 2 > t: print('Средние выборок не равны, x преобладает')
    elif t > 1 - alpha / 2: print('Средние выборок не равны, y преобладает')
    print()

def sign_test_big(x, y):
    minus, plus = 0, 0
    for (x_i, y_i) in zip(x, y):
        if x_i < y_i: minus += 1
        elif x_i > y_i: plus += 1
    nk = minus + plus
    w = min(minus, plus)
    t = (w - nk / 2) / (n / 4) ** 0.5

    u_half_alpha = - sps.norm.ppf(alpha / 2)
    print(f"Статистика t = {t}, интервал [{- u_half_alpha}, {u_half_alpha}]")
    if - u_half_alpha < t < u_half_alpha: print('Средние выборок равны')
    elif - u_half_alpha > t: print('Средние выборок не равны, x преобладает')
    elif t > u_half_alpha: print('Средние выборок не равны, y преобладает')
    print()

In [118]:
gauss_exp = 0
gauss_var = 7
n = 20
a, b = -10, 10

x = sps.norm(gauss_exp, gauss_var ** 0.5).rvs(n)
y1 = 5 * x + sps.uniform(a, b - a).rvs(n)
print("Равномерный шум:")
print()
sign_test_small(x, y1)

Равномерный шум:

Число '-': 9, число '+': 11
Статистика t = 0.41190147399902344, интервал [-0.025, 0.975]
Средние выборок равны



In [119]:
exp_lambda = 20
y2 = 5 * x + np.random.exponential(1 / exp_lambda, n)
print("Экспоненциальный шум:")
print()
sign_test_small(x, y2)

Экспоненциальный шум:

Число '-': 10, число '+': 10
Статистика t = 0.5880985260009766, интервал [-0.025, 0.975]
Средние выборок равны



# Ранговый критерий знаков

In [298]:
def signed_rang_test(x, y):
    dif = abs(x - y)
    np.delete(dif, np.where(dif == 0))
    n = len(dif)
    k = n // 4
    a, b = 0, max(dif)
    bounds = [a + i * (b - a) / k for i in range(k + 1)]
    ints = [(bounds[i], bounds[i + 1]) for i in range(len(bounds) - 1)]
    t = sum([np.sign(d) * [j for j in range(k) if ints[j][0] <= d <= ints[j][1]][0] for d in dif])

    # dif_sort = sorted(dif)
    # t = sum([np.sign(d) * list(dif_sort).index(d) for d in dif])
    t = t / (n * (n + 1) * (2 * n + 1) / 6) ** 0.5

    u_half_alpha = - sps.norm.ppf(alpha / 2)
    print(f"Статистика t = {t}, интервал [{- u_half_alpha}, {u_half_alpha}]")
    if - u_half_alpha < t < u_half_alpha: print('Средние выборок равны')
    elif - u_half_alpha > t: print('Средние выборок не равны, x преобладает')
    elif t > u_half_alpha: print('Средние выборок не равны, y преобладает')
    print()

In [299]:
n = 100

x = sps.norm(gauss_exp, gauss_var ** 0.5).rvs(n)
y1 = 5 * x + sps.uniform(a, b - a).rvs(n)
print("Равномерный шум:")
print()
print("Ассимптотический критерий знаков")
sign_test_big(x, y1)
print("Ранговый критерий знаков")
signed_rang_test(x, y1)

Равномерный шум:

Ассимптотический критерий знаков
Статистика t = -0.2, интервал [-1.9599639845400545, 1.9599639845400545]
Средние выборок равны

Ранговый критерий знаков
Статистика t = 1.086510650579627, интервал [-1.9599639845400545, 1.9599639845400545]
Средние выборок равны



In [300]:
n = 500
x = sps.norm(gauss_exp, gauss_var ** 0.5).rvs(n)

exp_lambda = 20
y2 = 5 * x + np.random.exponential(scale=1/exp_lambda, size=n)
print("Экспоненциальный шум:")
print()
print("Ассимптотический критерий знаков")
sign_test_big(x, y2)
print("Ранговый критерий знаков")
signed_rang_test(x, y2)

Экспоненциальный шум:

Ассимптотический критерий знаков
Статистика t = -0.35777087639996635, интервал [-1.9599639845400545, 1.9599639845400545]
Средние выборок равны

Ранговый критерий знаков
Статистика t = 2.460920608434264, интервал [-1.9599639845400545, 1.9599639845400545]
Средние выборок не равны, y преобладает



# Коэффициенты корреляции

In [140]:
def kendall_cor_coef(x, y):
    consist, inconsist = 0, 0
    n = len(x)
    for i in range(n):
        for j in range(i, n):
            if (x[i] < x[j] and y[i] < y[j]) or (x[i] > x[j] and y[i] > y[j]): consist += 1
            else: inconsist += 1
    r = (consist - inconsist) / math.comb(n, 2)
    t = r / (2 * (2 * n + 5) / (9 * n * (n - 1))) ** 0.5

    u_half_alpha = - sps.norm.ppf(alpha / 2)
    print(f"Статистика t = {t}, интервал [{- u_half_alpha}, {u_half_alpha}]")
    if - u_half_alpha < t < u_half_alpha: print('Выборки не коррелированы')
    else: print("Выборки коррелированы")
    print()

In [189]:
def pirson_cor_coef(x, y):
    n = len(x)
    x_mean = sum(x) / n
    y_mean = sum(y) / n
    r = sum([(x[i] - x_mean) * (y[i] - y_mean) for i in range(len(x))])
    r = r / (sum([(x_i - x_mean) ** 2 for x_i in x]) * sum([(y_i - y_mean) ** 2 for y_i in y])) ** 0.5
    t = 0.5 * np.log((1 + r) / (1 - r)) / (1 / (n - 3)) ** 0.5
    print(f"Коэффициент корреляции: r = {r}")

    u_half_alpha = - sps.norm.ppf(alpha / 2)
    # print(f"Статистика Фишера t = {t}, интервал [{- u_half_alpha}, {u_half_alpha}]")
    print(f"Статистика t = {t}, интервал [{- u_half_alpha}, {u_half_alpha}]")
    if - u_half_alpha < t < u_half_alpha: print('Выборки не коррелированы')
    else: print("Выборки коррелированы")

    # t = r * ((n - 2) / (1 - r ** 2)) ** 0.5
    # tau_half_alpha = sps.t.ppf(1 - alpha / 2, n - 2)
    # print(f"Статистика Стьюдента t = {t}, интервал [{- tau_half_alpha}, {tau_half_alpha}]")
    # if - tau_half_alpha < t < tau_half_alpha: print('Выборки не коррелированы')
    # else: print("Выборки коррелированы")
    print()

In [190]:
n = 100

gauss_exp = 0
gauss_var = 7
a, b = -10, 10
x = sps.norm(gauss_exp, gauss_var ** 0.5).rvs(n)
y1 = 5 * x + sps.uniform(a, b - a).rvs(n)
print("Равномерный шум:")
print()
# 0/5 ошибки
print("Коэффициент Кендала")
kendall_cor_coef(x, y1)
# 0/5 ошибки
print("Коэффициент Пирсона")
pirson_cor_coef(x, y1)

Равномерный шум:

Коэффициент Кендала
Статистика t = 10.35789018744913, интервал [-1.9599639845400545, 1.9599639845400545]
Выборки коррелированы

Коэффициент Пирсона
Коэффициент корреляции: r = 0.9161268527794637
Статистика t = 15.407334786836316, интервал [-1.9599639845400545, 1.9599639845400545]
Выборки коррелированы



In [191]:
exp_lambda = 20
y2 = 5 * x + np.random.exponential(1 / exp_lambda, n)
print("Равномерный шум:")
print()
# 0/5 ошибки
print("Коэффициент Кендала")
kendall_cor_coef(x, y2)
# 0/5 ошибки
print("Коэффициент Пирсона")
pirson_cor_coef(x, y2)

Равномерный шум:

Коэффициент Кендала
Статистика t = 14.420041485804683, интервал [-1.9599639845400545, 1.9599639845400545]
Выборки коррелированы

Коэффициент Пирсона
Коэффициент корреляции: r = 0.9999935059256584
Статистика t = 62.23377210700012, интервал [-1.9599639845400545, 1.9599639845400545]
Выборки коррелированы



# Проверка автокорреляции

In [24]:
def autocorrelation_coef_tests(x):
    n = len(x)
    r = n * sum([x[i] * x[i + 1] for i in range(n - 1)]) - (sum(x)) ** 2 + n * x[0] * x[n - 1]
    r = r / (n * sum([x_i ** 2 for x_i in x]) - (sum(x)) ** 2)
    print(f"Коэффициент корреляции: r = {r}")
    t = (r + 1 / (n - 1)) / (n * (n - 3) / ((n + 1) * (n - 1) ** 2)) ** 0.5
    u_half_alpha = - sps.norm.ppf(alpha / 2)
    print(f"Статистика t = {t}, интервал [{- u_half_alpha}, {u_half_alpha}]")
    if - u_half_alpha < t < u_half_alpha: print('Автокорреляции нет')
    else: print("Выборки автокоррелированы")
    print()

def moran_test(x):
    n = len(x)
    r = n * sum([x[i] * x[i + 1] for i in range(n - 1)]) - sum(x) ** 2 + n * x[0] * x[n - 1]
    r = r / (n * sum([x_i ** 2 for x_i in x]) - sum(x) ** 2)
    t = ((n * r + 1)/ (n - 2)) * (n - 1) ** 0.5
    u_half_alpha = - sps.norm.ppf(alpha / 2)
    print(f"Статистика t = {t}, интервал [{- u_half_alpha}, {u_half_alpha}]")
    if - u_half_alpha < t < u_half_alpha: print('Автокорреляции нет')
    else: print("Выборки автокоррелированы")
    print()

def ljung_box_test(x):
    n = len(x)
    r = n * sum([x[i] * x[i + 1] for i in range(n - 1)]) - sum(x) ** 2 + n * x[0] * x[n - 1]
    r = r / (n * sum([x_i ** 2 for x_i in x]) - sum(x) ** 2)
    t = r * (n * (n + 2) / (n - 1)) ** 0.5
    u_half_alpha = - sps.norm.ppf(alpha / 2)
    print(f"Статистика t = {t}, интервал [{- u_half_alpha}, {u_half_alpha}]")
    if - u_half_alpha < t < u_half_alpha: print('Автокорреляции нет')
    else: print("Выборки автокоррелированы")
    print()

def duff_roy_test(x):
    n = len(x)
    r = n * sum([x[i] * x[i + 1] for i in range(n - 1)]) - sum(x) ** 2 + n * x[0] * x[n - 1]
    r = r / (n * sum([x_i ** 2 for x_i in x]) - sum(x) ** 2)
    t = n * r * ((n - 1) / (n * (n - 2))) ** 0.5
    u_half_alpha = - sps.norm.ppf(alpha / 2)
    print(f"Статистика t = {t}, интервал [{- u_half_alpha}, {u_half_alpha}]")
    if - u_half_alpha < t < u_half_alpha: print('Автокорреляции нет')
    else: print("Выборки автокоррелированы")
    print()

In [99]:
def wald_wolfitz_test(x):
    n = len(x)
    S1 = sum(x)
    S2 = sum([x_i ** 2 for x_i in x])
    S3 = sum([x_i ** 3 for x_i in x])
    S4 = sum([x_i ** 4 for x_i in x])
    R = sum([x[i] * x[i + 1] for i in range(n - 1)]) + x[0] * x[n - 1]
    M = (S1 ** 2 - S2) / (n - 1)
    D = (S2 ** 2 - S4) / (n - 1) + (S1 ** 4 - 4 * S1 ** 2 * S2 + 4 * S1 * S3 + S2 ** 2 - 2 * S4) / ((n - 1) * (n - 2)) + \
        (S1 ** 2 - S2) ** 2 / (n - 1) ** 2
    R = (R - M) / D ** 0.5
    u_half_alpha = - sps.norm.ppf(alpha / 2)
    print(f"Статистика R = {R}, интервал [{- u_half_alpha}, {u_half_alpha}]")
    if - u_half_alpha < R < u_half_alpha: print('Автокорреляции нет')
    else: print("Выборки автокоррелированы")
    print()

def rang_wald_wolfitz_test(x):
    n = len(x)
    x_row = sorted(x)
    R = sum([(list(x_row).index(x[i]) - (n + 1) / 2) * (list(x_row).index(x[i + 1]) - (n + 1) / 2)
             for i in range(n - 1)])
    R = R / (n ** 2 * (n + 1) * (n - 3) * (5 * n + 6) / 720) ** 0.5
    u_half_alpha = - sps.norm.ppf(alpha / 2)
    print(f"Статистика R = {R}, интервал [{- u_half_alpha}, {u_half_alpha}]")
    if - u_half_alpha < R < u_half_alpha: print('Автокорреляции нет')
    else: print("Выборки автокоррелированы")
    print()

def rang_duff_roy_test(x):
    n = len(x)
    x_row = sorted(x)
    R = sum([(list(x_row).index(x[i]) - (n + 1) / 2) * (list(x_row).index(x[i + 1]) - (n + 1) / 2)
             for i in range(n - 1)]) / sum([(list(x_row).index(x[i]) - (n + 1) / 2) ** 2
                                            for i in range(n)])
    R = (R + 1 / n) / ((5 * n ** 3 - 19 * n ** 2 + 10 * n + 16) / (5 * n ** 2 * (n - 1) ** 2)) ** 0.5
    u_half_alpha = - sps.norm.ppf(alpha / 2)
    print(f"Статистика R = {R}, интервал [{- u_half_alpha}, {u_half_alpha}]")
    if - u_half_alpha < R < u_half_alpha: print('Автокорреляции нет')
    else: print("Выборки автокоррелированы")
    print()

In [107]:
n = 100
gauss_exp = 5
gauss_var = 7

x = sps.norm(gauss_exp, gauss_var ** 0.5).rvs(n)
print("Исходная выборка:")
print("Коэффициент автокорреляции Пирсона")
autocorrelation_coef_tests(x)
print("Критерий Морана")
moran_test(x)
print("Критерий Люнга-Бокса")
ljung_box_test(x)
print("Критерий Дюффа-Роя")
duff_roy_test(x)
print("Критерий Вальда-Вольфовица")
wald_wolfitz_test(x)
print("Ранговый критерий Вальда-Вольфовица")
rang_wald_wolfitz_test(x)
print("Ранговый критерий Дюффа-Роя")
rang_duff_roy_test(x)

Исходная выборка:
Коэффициент автокорреляции Пирсона
Коэффициент корреляции: r = 0.016648614410530154
Статистика t = 0.270226355803009, интервал [-1.9599639845400545, 1.9599639845400545]
Автокорреляции нет

Критерий Морана
Статистика t = 0.2705615974977137, интервал [-1.9599639845400545, 1.9599639845400545]
Автокорреляции нет

Критерий Люнга-Бокса
Статистика t = 0.16898983563550024, интервал [-1.9599639845400545, 1.9599639845400545]
Автокорреляции нет

Критерий Дюффа-Роя
Статистика R = 1.1385607743743242, интервал [-1.9599639845400545, 1.9599639845400545]
Автокорреляции нет

Критерий Вальда-Вольфовица
Статистика R = 0.007364362528606569, интервал [-1.9599639845400545, 1.9599639845400545]
Автокорреляции нет

Ранговый критерий Вальда-Вольфовица
Статистика R = 0.011840689557721417, интервал [-1.9599639845400545, 1.9599639845400545]
Автокорреляции нет

Ранговый критерий Дюффа-Роя
Статистика R = 0.1128117720795883, интервал [-1.9599639845400545, 1.9599639845400545]
Автокорреляции нет



In [301]:
y = np.copy(x[:n - 1])
for i in range(len(x) - 1): y[i] += 0.1 * x[i + 1]
print("Выборка с автокорреляцией:")
print("Коэффициент автокорреляции Пирсона")
autocorrelation_coef_tests(y)
print("Критерий Морана")
moran_test(y)
print("Критерий Люнга-Бокса")
ljung_box_test(y)
print("Критерий Дюффа-Роя")
duff_roy_test(y)
print("Критерий Вальда-Вольфовица")
wald_wolfitz_test(y)
print("Ранговый критерий Вальда-Вольфовица")
rang_wald_wolfitz_test(y)
print("Ранговый критерий Дюффа-Роя")
rang_duff_roy_test(y)

Выборка с автокорреляцией:
Коэффициент автокорреляции Пирсона
Коэффициент корреляции: r = 0.07772308470267258
Статистика t = 1.7846418843637142, интервал [-1.9599639845400545, 1.9599639845400545]
Автокорреляции нет

Критерий Морана
Статистика t = 1.786342602833222, интервал [-1.9599639845400545, 1.9599639845400545]
Автокорреляции нет

Критерий Люнга-Бокса
Статистика t = 1.7414238698465987, интервал [-1.9599639845400545, 1.9599639845400545]
Автокорреляции нет

Критерий Дюффа-Роя
Статистика R = 36.655908612994345, интервал [-1.9599639845400545, 1.9599639845400545]
Выборки автокоррелированы

Критерий Вальда-Вольфовица
Статистика R = 1.7823465784110122, интервал [-1.9599639845400545, 1.9599639845400545]
Автокорреляции нет

Ранговый критерий Вальда-Вольфовица
Статистика R = 1.5916052068632238, интервал [-1.9599639845400545, 1.9599639845400545]
Автокорреляции нет

Ранговый критерий Дюффа-Роя
Статистика R = 1.6379729579323856, интервал [-1.9599639845400545, 1.9599639845400545]
Автокорреляции 

# Критерий Хсу

In [213]:
def median(sample): # выборочная медиана
    n = len(sample)
    if n % 2 == 1: return sorted(sample)[n // 2]
    else: return (sorted(sample)[n // 2 - 1] + sorted(sample)[n // 2]) / 2

def xu_test(x):
    n = len(x)
    m = median(x)
    s = sum([(x[i] - m) ** 2 for i in range(n)])
    H = sum([i * (x[i] - m) ** 2 for i in range(n)]) / (s * (n - 1))
    H = (H - 0.5) / ((n + 1) / (6 * (n - 1) * (n + 2))) ** 0.5

    u_half_alpha = - sps.norm.ppf(alpha / 2)
    print(f"Статистика H = {H}, интервал [{- u_half_alpha}, {u_half_alpha}]")
    if - u_half_alpha < H < u_half_alpha: print('Сдвиг дисперсии отсутсвует')
    else: print("В некоторый момент происходит сдвиг дисперсии")
    print()

In [245]:
n = 100
gauss_exp = 0
gauss_var = 7

# 0/5 ошибок
print("Исходная выборка")
x = sps.norm(gauss_exp, gauss_var ** 0.5).rvs(n)
xu_test(x)

# 1/5 ошибок
print("Выборка со сдвигом")
x1 = x[0:n//2]
x2 = x[n//2:n] * 1.5
x_shift = np.concatenate([x1, x2])
xu_test(x_shift)

Исходная выборка
Статистика H = 0.42967423759163137, интервал [-1.9599639845400545, 1.9599639845400545]
Сдвиг дисперсии отсутсвует

Выборка со сдвигом
Статистика H = 3.0401166226956367, интервал [-1.9599639845400545, 1.9599639845400545]
В некоторый момент происходит сдвиг дисперсии

